# INF8775 – Analyse et conception d’algorithmes
# TP2 – Automne 2024

NOM, Prénom, 1234567

NOM, Prénom, 1234567

Note finale :

 <u>**Date limite de remise :**</u>  12 novembre 23h59 (Groupe B1), 5 novembre 23h59 (Groupe B2)

# Instructions

## Rédaction et remise du rapport

- Ce notebook constitue à la fois le sujet du TP, votre code et votre rapport. Il contient déjà du code pour faciliter vos mesures et l'affichage de vos résultats, ainsi qu'un squelette pour votre rapport.

- Complétez directement le notebook, vous êtes libres de créer de nouvelles cellules de code ou de texte.

- Remettez le fichier du notebook sur Moodle avec le nom `NOM1_MATRICULE1_NOM2_MATRICULE2.ipynb`

- Vous pouvez vous inspirer de code trouvé sur Internet, tant que vous en mentionnez la source, mais vous devez créer votre propre implémentation, sous peine d'être sanctionnés pour plagiat.

## Mise en situation

Ce travail pratique se répartit sur deux séances de laboratoire et porte sur l'analyse et la conception d'algorithmes développés suivant différents patrons de conception afin de résoudre une version simplifiée d'un problème réaliste d'optimisation.


## Description du problème

Dans ce TP, vous allez travailler sur le problème de partition. De manière générale, celui-ci consiste à séparer un ensemble de poids en k ensembles de poids total identique. Dans ce cas-ci, on cherche, à partir d'une liste de valeurs, à obtenir deux listes dont la somme des valeurs est la même.

Par exemple :
- L'exemplaire est : `[6,3,5,2]`
- La seule solution est : `[6,2][5,3]`




## Algorithmes à implanter

Trois algorithmes seront implantés, mettant en pratique des patrons de conception différents :

1. Un algorithme glouton
2. Un algorithme de programmation dynamique
3. Un algorithme d'amélioration locale


## Jeu de données

La classe Problem existe pour simplifier l'interface des différentes fonctions utilitaires. Elle permet de générer des jeux de données avec la méthode `generate_sample` ci-dessous. Elle génère une liste d'une taille donnée contenant des nombres dont la somme est équivalente au total spécifié. Toutes les exemplaires devraient pouvoir être partionner, tant que la somme que vous donnez est pair.

In [1]:
import random
from collections.abc import Iterable, Callable


def simple(size: int, total: int) -> list[int]:
    """Samples generated using a uniform distribution capped at the average value.
    Simplest sample to solve"""
    instance = []
    current_total = 0

    for i in range(size - 1):
        max_val = round((total - current_total) / (size - i))
        value = random.randint(1, max_val)
        current_total += value
        instance.append(value)

    instance.append(int(total - current_total))
    return instance


def uniform(size: int, total: int) -> list[int]:
    """Samples generated using a uniform distribution around the average value"""
    instance = []
    current_total = 0

    for i in range(size - 1):
        avg_val = round((total - current_total) / (size - i))
        value = random.randint(round(avg_val*0.5), round(avg_val*1.5))
        current_total += value
        instance.append(value)
        
    instance.append(int(total - current_total))
    return instance


def normal(size: int, total: int) -> list[int]:
    """Samples generated using a normal distribution"""
    instance = []
    current_total = 0

    for i in range(size - 1):
        avg_val = round((total - current_total) / (size - i))
        value = round(random.normalvariate(avg_val,0.3*avg_val))
        current_total += value
        instance.append(value)
        
    instance.append(int(total - current_total))
    return instance


class Problem():
    def __init__(
        self,
        size: int, 
        total: int, 
        num_samples: int = 5, 
        gen_algo: Callable[[int,int],list[int]] = simple
    ) -> None:
        self.num_samples: int = num_samples
        self.size: int = size
        self.total: int = total
        self.gen_algo = gen_algo

    def generate_sample(self) -> list[int]:
        """Returns a list of the given size containing numbers between 1 and the max_number"""
        left_size = random.randint(2,self.size-1)
        instance = self.gen_algo(left_size,self.total/2)
        instance.extend(self.gen_algo(self.size - left_size,self.total/2))
        random.shuffle(instance)
        return instance

    def generate_dataset(self) -> Iterable[list[int]]:
        """Returns an iterator over as many samples as are described """
        return (self.generate_sample() for _ in range(self.num_samples))

# Implantations et expérimentations

Ces fonctions auxiliaires vous sont fournies pour vérifier l'exactitude de vos algorithmes, mesurer leur performance et afficher vos résultats.

Il est recommandé de prendre le temps de lire et de comprendre le code.

Exécutez la cellule ci-dessous pour pouvoir utiliser les fonctions auxiliaires.

In [2]:
import matplotlib.pyplot as plt
import time
from scipy.stats import linregress


class InvalidSolution(Exception):
    def __init__(self):
        super().__init__("Invalid solution, verify your code.")


class Measure():
    """A wrapper to contain information on taken measures. mae is Mean Absolute Error"""
    def __init__(self, size: int, total: int, mean_time: float, correct_frac:float, mae:float) -> None:
        """Args:
            mae (float): Mean Absolute Error
        """
        self.size = size
        self.total = total
        self.mean_time = mean_time
        self.correct_frac = correct_frac
        self.mae = mae


def make_problems(
    sizes: list[int],
    totals: list[int],
    num_samples: int = 5,
    gen_algo: Callable[[int,int],list[int]] = simple
) -> list[Problem]:
    """Creates problem instances using given sizes and total value"""
    problems: list[Problem] = []
    for size in sizes:
        for total in totals:
            problems.append(Problem(size,total,num_samples,gen_algo))
    return problems


def get_absolute_error(solution: list[list[int]]) -> int:
    """Check if solution is well partitioned """
    target = (sum(solution[0]) + sum(solution[1])) / 2
    return abs(sum(solution[0]) - target)


def get_list_freq(original: list[int]) -> dict[int,int]:
    """Converts a list into a dictionary of frequencies"""
    freq: dict[int,int] = dict()
    for iter in original:
        if iter not in freq.keys():
            freq[iter] = 0
        freq[iter] += 1
    return freq


def verify_solution_format(original: list[int], solution: list[list[int]]) -> bool:
    """Validates if the solution format is correct. This does not guarantee a solution
    with 0 error"""
    if len(solution) != 2:
        return False

    solution = [value for partition in solution for value in partition]

    # Lists must be of equal length
    if len(solution) != len(original):
        return False

    original_freq = get_list_freq(original)
    solution_freq = get_list_freq(solution)
    # Lists must have the same values
    for key in original_freq.keys():
        if key not in solution_freq.keys() or\
            solution_freq[key] != original_freq[key]:
            return False

    # Solution is valid
    return True


def measure(
    procedure: Callable[[list[int]],list[list[int]]],
    sample: list[int],
    time_scale: int = 1000
) -> tuple[int,int]:
    """Returns the time in milliseconds taken to run the procedure as well as the absolute error in a tuple.

    Raises:
        InvalidSolution: If the procedure returns an invalid solution, raises an exception.
    """
    sample = sample.copy()
    start: int = time.time() * time_scale
    solution: list[list[int]] = procedure(sample)
    end: int = time.time() * time_scale
    if not verify_solution_format(sample, solution):
        raise InvalidSolution()
    return (round(end - start), get_absolute_error(solution))


def measure_mean(procedure: Callable[[list[int]],list[list[int]]], prob: Problem, time_scale: int = 1000) -> Measure:
    """Generates multiple samples with the specified parameters and returns the mean time in milliseconds

    Raises:
        InvalidSolution: If one of the samples results in an invalid solution.
    """
    measures = [measure(procedure,sample,time_scale) for sample in prob.generate_dataset()]
    mean_time = sum(iter[0] for iter in measures) / prob.num_samples
    n = len(measures)
    n_correct = len([iter for iter in measures if iter[1] == 0])
    correct_frac = n_correct / n
    mae = 0
    if n_correct != n:
        mae = sum(iter[1] for iter in measures) / (n - n_correct)
    return Measure(prob.size, prob.total, mean_time, correct_frac, mae)


def measure_range(procedure: Callable[[list[int]],list[list[int]]], problems: list[Problem], time_scale: int = 1000) -> list[Measure]:
    """Measures the mean time taken in milliseconds for each size in the given list.

    Raises:
        InvalidSolution: If one of the samples results in an invalid solution.

    Returns:
        A list of Measure instances containing the specifications
        of the problem as well as the mean time.
    """
    return [
        measure_mean(procedure, prob, time_scale)
        for prob in problems
    ]


def display_data_as_table(measures: list[Measure]):
    """Prints a table with the data in the given list of measures"""
    print("{: <20} {: <20} {: <20} {: <20} {: <20}".format("Taille","Somme","Temps moyen (ms)", "Taux de succès", "Erreur absolue moyenne"))
    for measure in measures:
        print("{: <20} {: <20} {: <20} {: <20} {: <20}".format(measure.size, measure.total, measure.mean_time, measure.correct_frac, round(measure.mae, 3)))


### The different tests are below, the names are in french to avoid confusion


def test_de_puissance(
    data: dict[int,int],
    x_label: str,
    y_label: str,
    title: str = "Test de puissance"
):
    """Takes the data and displays it into the corresponding test graph.
    It applies no transformations to the data.

    Args:
        data (dict[int,int]): A dictionnary mapping the x variable to the y variable
    """
    # Log both sets of values
    x = list(data.keys())
    y = list(data.values())

    # Perform the lin regression
    m, b, rvalue, _, _ = linregress(x, y)

    # Estimate the values of y based on the lin regression results
    predicted = [m * iter + b for iter in x]

    # Create the line equation
    line_eq = f"y = {m:.2f}x + {b:.2f}"

    # Plot the points
    plt.scatter(x, y, label='Mesures')

    # Plot the regression line
    plt.plot(x, predicted, color="red", label=f'Regression linéaire R²={round(rvalue**2,6)}')

    # Add labels and title
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)

    # Add legend
    plt.legend(bbox_to_anchor=(0.60, 0), loc='lower left')

    # Display the line equation
    plt.text(min(x), max(y), line_eq)

    # Show the plot
    plt.show()


def test_de_rapport(
    data: dict[int,int],
    x_label: str,
    y_label: str,
    title: str = "Test de rapport"
):
    """Takes the data and displays it into the corresponding test graph.
    It applies no transformations to the data.

    Args:
        data (dict[int,int]): A dictionnary mapping the x variable to the y variable
    """
    x = list(data.keys())
    y = list(data.values())

    plt.plot(x, y, label='Mesures')
    plt.scatter(x, y, label='Mesures')

    # Add labels and title
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    plt.show()


def test_de_constantes(
    data: dict[int,int],
    x_label: str,
    y_label: str = "Temps (ms)",
    title: str = "Test de constantes"
):
    """Takes the data and displays it into the corresponding test graph.
    It applies no transformations to the data.

    Args:
        data (dict[int,int]): A dictionnary mapping the x variable to the y variable
    """
    x = list(data.keys())
    y = list(data.values())

    # Perform linear regression
    m, b, rvalue, _, _ = linregress(x, y)

    predicted = [m * iter + b for iter in x]

    # Create the line equation
    line_eq = f"y = {m:.2E}x + {b:.2E}"

    # Plot the points
    plt.scatter(x, y, label='Mesures')

    # Plot the regression line
    plt.plot(x, predicted, color="red", label=f'Regression linéaire R²={round(rvalue**2,6)}')

    # Add labels and title
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)

    # Add legend
    plt.legend(bbox_to_anchor=(0.60, 0), loc='lower left')

    # Display the line equation
    plt.text(min(x), max(y), line_eq)

    # Show the plot
    plt.show()

## Partie 1 : Algorithme glouton (5 points)

<u>**Question 1 :**</u> Implantez un algorithme suivant le patron de conception glouton.

Il est normal que le glouton ne retourne pas toujours la solution optimale. Vous pouvez voir l'erreur obtenue et le taux de réponse optimal avec les fonctions de mesure.

Tentez de rendre votre implantation la plus performante possible en évitant des calculs inutiles.

In [11]:
def glouton_partition(liste):
    liste1 = []
    liste2 = []
    somme1 = 0
    somme2 = 0
    tailleListe = len(liste) - 1
    liste = sorted(liste, reverse=True)

    for i in range(tailleListe + 1):
        if somme1 < somme2:
            liste1.append(liste[i])
            somme1 += liste[i]
        else:
            liste2.append(liste[i])
            somme2 += liste[i]
    return [liste1, liste2]

### Analyse asymptotique

<u>**Question 2 :**</u> Quelle est la complexité asymptotique théorique de cet algorithme? Expliquez

Complexité en O(n).

### Mesures

<u>**Question 3 :**</u> Rapportez dans un tableau les temps d'exécution moyens avec les fonctions auxiliaires `measure_range` et `display_data_as_table`.

In [30]:
list_problems = make_problems(range(10, 100000, 5000),[10000000])
list_measure = measure_range(glouton_partition, list_problems)
display_data_as_table(list_measure)

Taille               Somme                Temps moyen (ms)     Taux de succès       Erreur absolue moyenne
10                   10000000             0.0                  0.2                  28423.0             
5010                 10000000             1.0                  0.8                  1.0                 
10010                10000000             1.2                  1.0                  0                   
15010                10000000             2.0                  1.0                  0                   
20010                10000000             3.0                  1.0                  0                   
25010                10000000             3.0                  1.0                  0                   
30010                10000000             4.0                  1.0                  0                   
35010                10000000             4.0                  1.0                  0                   
40010                10000000             5.0        

### Validation empirique

<u>**Question 4 :**</u> Servez-vous de vos temps d'exécution pour confirmer et/ou préciser l'analyse asymptotique théorique de vos algorithmes avec la méthode hybride de votre choix.

La méthode peut varier d'un algorithme à l'autre. Justifiez les choix ici et avec des graphiques.

*Insérer votre réponse ici*

## Partie 2 : Algorithme de programmation dynamique (5 points)

Une solution au problème de partition requiert nécessairement que la somme des valeurs dans chacune des partitions soit la moitié de la somme de l'exemplaire complet. Ainsi, il suffit de trouver une ensemble de valeurs égalisant la moitié de la somme totale pour trouver une réponse valide. Toutes les autres valeurs formeront la seconde partition.

Considérez un sous ensemble *X*. Les membres de *X* peuvent être combinés pour obtenir différentes valeurs. En ajoutant une nouvelle variable *y* au sous-ensemble, pour chacune des valeurs atteignables précédemment, il est possible d'atteindre la valeur + *y*.

Ainsi, l'algorithme de programmation dynamique répondant à ce problème consiste à explorer les valeurs obtenables à partir d'un sous-ensemble de valeurs en ajoutant progressivement les valeurs de l'exemplaire jusqu'à obtenir un sous-ensemble dont la valeur combinée est la moitié de la valeur de l'instance.

<u>**Question 1 :**</u> Implantez l'algorithme de programmation dynamique.

In [41]:
def dynamique_partition(liste_entiers):
    cible = sum(liste_entiers) // 2

    # On initialise un tableau que l'on va remplir pour que Tab[i][j] soit True si la valeur j est atteignable avec les i premiers termes.
    n = len(liste_entiers)
    Tab = [[False] * (cible + 1) for _ in range(n + 1)]
    
    for i in range(n + 1):
        Tab[i][0] = True

    # On remplit Tab
    for i in range(1, n + 1):
        for j in range(1, cible + 1):
            if liste_entiers[i - 1] <= j:
                Tab[i][j] = Tab[i - 1][j] or Tab[i - 1][j - liste_entiers[i - 1]]
            else:
                Tab[i][j] = Tab[i - 1][j]

    # Si la somme cible n'est pas atteignable, on renvoit deux listes vides
    if not Tab[n][cible]:
        return [[], []]

    # Sinon on reconstitue les deux listes. Pour chaque élément i, on regarde si cible est atteignable avec les i-1 premier termes. Si non, l'élément a été utilisé.
    liste_variable_utilise_atteindre_cible = []
    liste_reste = liste_entiers[:]

    for i in range(n, 0, -1):
        if not Tab[i - 1][cible]:
            liste_variable_utilise_atteindre_cible.append(liste_entiers[i - 1])
            liste_reste.remove(liste_entiers[i - 1])
            cible -= liste_entiers[i - 1]

    return [liste_variable_utilise_atteindre_cible, liste_reste]

### Analyse asymptotique

<u>**Question 2 :**</u> Quelle est la complexité asymptotique théorique de cet algorithme? Expliquez

*Insérer votre réponse ici*

### Mesures

<u>**Question 3 :**</u> Rapportez dans un tableau les temps d'exécution moyens avec les fonctions auxiliaires `measure_range` et `display_data_as_table` **C'est attendu que cet algorithme soit beaucoup plus lent que le glouton. Vous n'avez pas besoin de mesurer sur des exemplaires aussi gros.**

### Validation empirique

<u>**Question 4 :**</u> Servez-vous de vos temps d'exécution pour confirmer et/ou préciser l'analyse asymptotique théorique de vos algorithmes avec la méthode hybride de votre choix.

La méthode peut varier d'un algorithme à l'autre. Justifiez les choix ici et avec des graphiques.

*Insérer votre réponse ici*

## Partie 3 : Algorithme d'amélioration locale (5 points)

Cet algorithme explore le voisinage de solutions non optimales. En partant d’une solution, celle-ci est améliorée en modifiant une ou plusieurs de ses assignations. Vous pouvez prendre
comme critère d’arrêt un nombre maximum d’itérations et comme fonction de coût `get_absolute_error`.

<u>**Question 1 :**</u> Implantez un algorithme suivant le patron de conception d'amélioration locale.

In [47]:
def amelioration_locale_partition(liste_entiers, max_iterations=100000):
    S1 = random.sample(liste_entiers, len(liste_entiers) // 2)
    S2 = [x for x in liste_entiers if x not in S1]
    somme_totale = sum(liste_entiers)
    
    meilleur_cout = get_absolute_error([S1, S2])
    
    for iteration in range(max_iterations):
        if S1 and S2:  
            elem_S1 = random.choice(S1)
            elem_S2 = random.choice(S2)
            S1.remove(elem_S1)
            S2.remove(elem_S2)
            S1.append(elem_S2)
            S2.append(elem_S1)
            
            cout_actuel = get_absolute_error([S1, S2])
            
            if cout_actuel < meilleur_cout:
                meilleur_cout = cout_actuel
            else:
                S1.remove(elem_S2)
                S2.remove(elem_S1)
                S1.append(elem_S1)
                S2.append(elem_S2)

            if meilleur_cout == 0:
                break
    
    return S1, S2, meilleur_cout

### Analyse asymptotique

<u>**Question 2 :**</u> Quelle est la complexité asymptotique théorique de cet algorithme? Expliquez

*Insérer votre réponse ici*

### Mesures

<u>**Question 3 :**</u> Rapportez dans un tableau les temps d'exécution moyens avec les fonctions auxiliaires `measure_range` et `display_data_as_table`. Vous pouvez comparer vos résultats à ceux sans l'amélioration locale pour voir l'amélioration.

### Validation empirique

<u>**Question 4 :**</u> Servez-vous de vos temps d'exécution pour confirmer et/ou préciser l'analyse asymptotique théorique de vos algorithmes avec la méthode hybride de votre choix.

La méthode peut varier d'un algorithme à l'autre. Justifiez les choix ici et avec des graphiques.

*Insérer votre réponse ici*

# Évaluation de la consommation énergétique (1 point)

Dans le cadre de ce TP, nous voulons vous sensibiliser à la consommation énergétique de vos algorithmes. Pour ce faire, nous vous fournissons une librairie que vous pouvez utiliser pour évaluer l'énergie nécessaire à la complétion de votre algorithme (https://mlco2.github.io/codecarbon/). Appelez chacun de vos algorithmes ci-bas avec les mêmes exemplaires et comparez le résultat obtenu par chacun.

In [49]:
#Commande pour installer la librairie
#!pip install codecarbon

In [ ]:
from codecarbon import EmissionsTracker
exemplaire = simple(100, 100000)
try:
    tracker = EmissionsTracker(measure_power_secs=5, tracking_mode="process")

    tracker.start_task("glouton")
    glouton_partition(exemplaire)
    tracker.stop_task()

    tracker.start_task("dynamic")
    dynamique_partition(exemplaire)
    tracker.stop_task()

    tracker.start_task("recherche")
    amelioration_locale_partition(exemplaire)
    tracker.stop_task()
finally:
    _ = tracker.stop()

Les résultats de l'exécution du code ci-haut devraient apparaître dans un fichier csv. Commentez les résultats obtenus. Vous pouvez trouver une explication de la sortie dans la documentation qui est présente au lien ci-haut.

*Insérer votre réponse ici*

# Conclusion et synthèse (2 points)

Résumez succinctement vos résultats et indiquez sous quelles conditions vous utiliseriez chacun des algorithmes.

*Insérer votre réponse ici*

## Autres critères (2 pts)
 Qualité du code / 1 pt

Présentation générale / 1 pt

- Concision
- Qualité du français

Pénalité retard
- -1 pt / journée de retard, arrondi vers le haut. Les TPs ne sont plus acceptés après 3 jours.